## Model1 & 2:
### 경형·소형 트럭: 경형 1톤 이하, 소형 2톤 미만 차급

- __Model1__: 카고형
- __Model2__: 박스형(윙바디/탑, 냉동, 냉장)
- __archive__: 포터 봉고 순정특장차(홈로리, 활어차, 덤프)
- 1톤 Base ( t >= 0.5 & t < 2)
- 소형트럭은 적재중량 1톤 이상 2톤 미만 차급을 지칭한다.<br>
  연간 신규등록대수는 약 15만대로, 트럭 시장에서 가장 많은 등록대수를 차지한다.
  용도 또한 용달, 개별 등을 비롯해, 농어촌 자가용 역할 등 많은 역할을 수행한다.
  대표모델로는 현대차 포터와 기아차 봉고가 있으며, 이외 최근 몇몇 업체서 소형급 전기트럭을 준비하고 있다.

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from hyperopt import STATUS_OK, Trials, fmin, hp as hps, tpe
import pickle
# modules 폴더를 path에 추가(본인 컴퓨터에 맞게 수정)
import sys
import os
root_path = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
module_path = root_path + '\\modules'
sys.path.insert(1, module_path)
from common import common as cm
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import glob
from datetime import datetime
import matplotlib
import json
matplotlib.rcParams['axes.unicode_minus'] = False
font_path = "../../../assets/fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
# 하나의 cell에서 multiple output 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# float > 소수점 두자리까지만
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.float_format = '{:,.4f}'.format
# 컬럼 길이 제한 없음
try:
    pd.set_option('max_columns', None)
    pd.set_option('max_rows', 500)
except:
    pd.options.display.max_columns = None
    pd.options.display.max_rows = 500

# 아래 코드에서 100%를 본인이 원하는 비율로 조정하여, 가로로 넓게 코드작성이 가능하고, 데이터를 보는데 쾌적한 환경을 조성할 수 있음
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [ ]:
# call 아이트럭 raw data
from calldb import db_helper
itruck = db_helper(server="REL").table("SELECT * FROM TB_MYCAR;")

max_workers = min(32, os.cpu_count() + 4)
# call 특장차8949 raw data
tjc = cm.read_all_csv(path=root_path+"/assets/csv/특장차8949/", workers=max_workers)

# call 카매니저 raw data
cmg = cm.read_all_json(path=root_path+"/assets/json/카매니저/metaData/", workers=max_workers)

print(f"itruck shape: {itruck.shape}\ntjc shape: {tjc.shape}\ncmg shape: {cmg.shape}")

In [ ]:
def scale(df):
    df = df.drop(columns=['id'])
    
    df = df[df['f_brand'].isin(['현대', '기아'])]
    df.loc[df['f_brand'] == '기아', 'f_brand'] = 0
    df.loc[df['f_brand'] == '현대', 'f_brand'] = 1
    df['f_year'] = pd.to_datetime(df['f_year'])
    df['f_reg_dt'] = pd.to_datetime(df['f_reg_dt'])
    df['f_category'] = df['f_category'].replace({
        '내장탑': 'top',
        '냉동냉장차': 'refrige',
        '윙바디': 'wing',
    })
    
    df.loc[df['f_trm'] == '수동', 'f_trm'] = 0
    df.loc[df['f_trm'] == '오토', 'f_trm'] = 1
    df.loc[df['f_elec_yn'] == False, 'f_elec_yn'] = 0
    df.loc[df['f_elec_yn'] == True, 'f_elec_yn'] = 1
    df = pd.get_dummies(df, columns=["f_category"])
    
    y = df.pop('l_price')
    X = df
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    for col in X:
        X[col] = scaler.fit_transform(X[[col]])
        
    return X, y

# ../../../assets/csv/시세예측/raw/model2/ 경로에서 가장 최근의 파일을 불러옴
df = pd.read_csv(sorted(glob.glob("../../../assets/csv/시세예측/raw/model2/*.csv"))[-1], encoding='cp949')
scaled_df = df.copy()
X, y = scale(scaled_df)
print(X.shape, y.shape)

In [ ]:
# split train, test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=200)

## 하이퍼 파라미터 튜닝

### Setting

In [ ]:
# regularization candiate 정의
reg_candidate = [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 5, 10, 100]

# space 정의, Hyperparameter의 이름을 key 값으로 입력(xgboost)
space={
    'max_depth': hps.quniform("max_depth", 5, 20, 1), # 트리의 최대 깊이
    'learning_rate': hps.quniform ('learning_rate', 0.01, 0.05, 0.005), # 학습률
    'reg_alpha': hps.choice('reg_alpha', reg_candidate), # L1정규화 Lasso회귀분석
    'reg_lambda': hps.choice('reg_lambda', reg_candidate), # L2정규화 Ridge회귀분석
    'subsample': hps.quniform('subsample', 0.01, 1, 0.01), # 트리를 생성할 때 데이터를 샘플링하는 비율
    'colsample_bytree': hps.quniform('colsample_bytree', 0.01, 1, 0.01), # 트리를 생성할 때 피처를 샘플링하는 비율
    'colsample_bylevel': hps.quniform('colsample_bylevel', 0.01, 1, 0.01), # 트리의 레벨별로 피처를 샘플링하는 비율
    'colsample_bynode': hps.quniform('colsample_bynode', 0.01, 1, 0.01), # 트리의 노드별로 피처를 샘플링하는 비율
    'min_child_weight': hps.quniform('min_child_weight', 1, 100, 1), # 트리의 리프노드가 되기 위한 최소한의 샘플 데이터 수
    'n_estimators': hps.quniform('n_estimators', 100, 10000, 100), # 트리의 개수
    'gamma': hps.quniform('gamma', 0.01, 1, 0.01), # 트리의 리프노드를 추가적으로 나눌지를 결정하는 파라미터
}

def hyperparameter_tuning(space):
    model = XGBRegressor(
        n_estimators =int(space['n_estimators']), 
        max_depth = int(space['max_depth']), 
        learning_rate = space['learning_rate'],
        reg_alpha = space['reg_alpha'], # L1정규화 Lasso회귀분석
        reg_lambda = space['reg_lambda'], # L2정규화 Ridge회귀분석
        subsample = space['subsample'], # 각 트리마다의 관측 데이터 샘플링 비율
        colsample_bytree = space['colsample_bytree'], # 각 트리마다의 feature 샘플링 비율
        colsample_bylevel = space['colsample_bylevel'], # 각 트리 depth 마다 사용할 feature 비율
        colsample_bynode = space['colsample_bynode'], # 각 트리 node 마다 사용할 feature 비율
        min_child_weight = int(space['min_child_weight']), # child의 관측에서 요구되는 최소 가중치의 합(overfitting 조정 parameter)
        gamma = space['gamma'], # 트리의 리프노드를 추가적으로 나눌지를 결정하는 파라미터
        random_state=200, 
    )
    
    evaluation = [(X_train, y_train), (X_test, y_test)]
    model.fit(
        X_train, y_train,
        eval_set=evaluation, 
        eval_metric="rmse",
        early_stopping_rounds=20,
        verbose=0,
    )
    pred = model.predict(X_test)
    rmse= cm.RMSE(y_test, pred)
    # 평가 방식 선정
    
    return {'loss':rmse, 'status': STATUS_OK, 'model': model}

### Run

In [ ]:
# Trials 객체 선언합니다.
trials = Trials()

# best에 최적의 하이퍼 파라미터를 return 받습니다.
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=100, # 최대 반복 횟수를 지정합니다.
            trials=trials
        )

# 최적화된 결과를 int로 변환해야하는 파라미터는 타입 변환을 수행합니다.
best['max_depth'] = int(best['max_depth'])
best['min_child_weight'] = int(best['min_child_weight'])
best['n_estimators'] = int(best['n_estimators'])
best['reg_alpha'] = reg_candidate[int(best['reg_alpha'])]
best['reg_lambda'] = reg_candidate[int(best['reg_lambda'])]
best['random_state'] = 200
print(best)

## Cross Validation, Get Best Model

In [ ]:
from sklearn.model_selection import KFold
import time

def kfold_xgb(xgb: XGBRegressor, X, y, n_splits=5, random_state=100):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    result = []
    for i, (train_idx, test_idx) in enumerate(kf.split(X)):
        d = {}
        loss = {
            'MAE': "",
            'RMSE': "",
            'RMSLE': "",
            'MAPE': "",
            'R2': "",
        }
        print(f'Fold {i+1}: ')
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        now = time.time()
        xgb.fit(X_train, y_train)
        print(f'학습 시간: {time.time()-now:.2f}초')
        now = time.time()
        pred = xgb.predict(X_test)
        TT_Sec = time.time()-now
        print(f'예측 시간: {TT_Sec:.2f}초')
        print('--------------------------------------')
        loss['MAE'], loss['RMSE'], loss['RMSLE'], loss['MAPE'], loss['R2'], loss['TT(Sec)'] = cm.MAE(y_test, pred), cm.RMSE(y_test, pred), cm.RMSLE(y_test, pred), cm.MAPE(y_test, pred), cm.R2(y_test, pred), TT_Sec
        result.append({"model": xgb, "X_train": X_train, "X_test": X_test, "y_train": y_train, "y_test": y_test, "pred": pred, "loss": loss})
        
    return result

xgb = XGBRegressor(**best)
result = kfold_xgb(xgb, X, y, n_splits=10, random_state=200)
best_result = result[np.argmin([r['loss']['RMSLE'] for r in result])]

## Visualize

In [ ]:
pd.DataFrame([r['loss'] for r in result], index=[f'Fold {i+1}' for i in range(len(result))])

print(f'Best: Fold {np.argmin([r["loss"]["RMSLE"] for r in result])+1}')
pred_df = pd.DataFrame({
    '실제값': best_result['y_test'],
    '예측값': best_result['pred'],
})
pred_df['오차값'] = (pred_df['예측값'] - pred_df['실제값'])
pred_df['절대 오차값'] = (pred_df['예측값'] - pred_df['실제값']).abs()
pred_df.describe()

In [ ]:
from sklearn.linear_model import LinearRegression

def show_plot():
    dfs = {f"Fold {i+1}": pd.DataFrame({
        '실제값': r['y_test'],
        '예측값': r['pred'],
        '오차값': (r['pred'] - r['y_test']),
    }) for i, r in enumerate(result)}

    y_len = len(result)
    figure, axis = plt.subplots(y_len, 2, figsize=(20, 10*y_len))

    best_fold = f'Fold {np.argmin([r["loss"]["RMSLE"] for r in result])+1}'
    for i, (key, df) in enumerate(dfs.items()):
        # grid true
        axis[i][0].grid(True)
        x_line = np.arange(0, df[['실제값', '예측값']].max().max())
        y_line = x_line + 0
        axis[i][0].axis([df['실제값'].min(), df['실제값'].max(), df['예측값'].min(), df['예측값'].max()])
        axis[i][0].scatter(df['실제값'], df['예측값'], alpha=.5, s=10)
        axis[i][0].plot(x_line, y_line, color='red')  # draw line
        axis[i][0].set_xlabel(df['실제값'].name)
        axis[i][0].set_ylabel(df['예측값'].name)
        if key == best_fold:
            axis[i][0].set_title(key + ' (Best)')
        else:
            axis[i][0].set_title(key)
        
        # grid true
        axis[i][1].grid(True)
        axis[i][1].axis([df['실제값'].min(), df['실제값'].max(), df['오차값'].min(), df['오차값'].max()])
        axis[i][1].scatter(df['실제값'], df['오차값'], alpha=.5, s=10)
        axis[i][1].plot([df['실제값'].min(), df['실제값'].max()], [0, 0], color='red')
        axis[i][1].set_xlabel(df['실제값'].name)
        axis[i][1].set_ylabel(df['오차값'].name)
        if key == best_fold:
            axis[i][1].set_title(key + ' (Best)')
        else:
            axis[i][1].set_title(key)
        # linear regression for 오차값
        x_r = df['실제값'].values.reshape(-1, 1)
        y_r = df['오차값'].values.reshape(-1, 1)
        lr = LinearRegression()
        lr.fit(x_r, y_r)
        axis[i][1].plot(x_r, lr.predict(x_r), color='green')
        
    plt.show();
    
show_plot()

## Save model

In [ ]:
save_dt = datetime.now()

model_path = "../../../assets/csv/시세예측/model/model2"
train_path = "../../../assets/csv/시세예측/train/model2"
test_path = "../../../assets/csv/시세예측/test/model2"
cm.create_folder(model_path)
cm.create_folder(train_path)
cm.create_folder(test_path)

# save xgBoost Model
best_result['model'].save_model(f"{model_path}/xg.{save_dt.strftime('%Y%m%d%H%I%S')}.json")
best_result['model'].save_model(f"{model_path}/xg.{save_dt.strftime('%Y%m%d%H%I%S')}.model")

# save minMaxScaler Model
mm_model = df.loc[best_result['X_train'].index][['f_mileage', 'f_year']].reset_index(drop=True)
transformed_datetime = cm.transform_date(mm_model['f_year'].values.reshape(-1, 1))
mm_model = pd.concat([mm_model, pd.DataFrame(transformed_datetime, columns=['transformed_datetime'])], axis=1)

scaler = MinMaxScaler(feature_range=(0, 1))

mm_fit = scaler.fit(mm_model.drop(columns=['f_year']))

with open(f"{model_path}/mm.{save_dt.strftime('%Y%m%d%H%I%S')}.model", 'wb') as f:
    pickle.dump(mm_fit, f)
    
# save test, train set
best_result['X_train'].to_csv(f"{train_path}/X_train.{save_dt.strftime('%Y%m%d%H%I%S')}.csv", index=False, encoding='cp949')
best_result['X_test'].to_csv(f"{test_path}/X_test.{save_dt.strftime('%Y%m%d%H%I%S')}.csv", index=False, encoding='cp949')
best_result['y_train'].to_csv(f"{train_path}/y_train.{save_dt.strftime('%Y%m%d%H%I%S')}.csv", index=False, encoding='cp949')
best_result['y_test'].to_csv(f"{test_path}/y_test.{save_dt.strftime('%Y%m%d%H%I%S')}.csv", index=False, encoding='cp949')